# Demo

In this demo we are going to load a session and create a psychometric curve. Let's start by loading all the packages:

In [2]:
import os
import matplotlib.pylab as plt
import numpy as np
import scipy.io
import math
import sys
import tables
#import pandas
import pickle as pkl
from scipy.stats import sem
from scipy.stats import pearsonr
from numpy.random import permutation
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from scipy.stats import ortho_group 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold,StratifiedKFold,StratifiedShuffleSplit
from sklearn.neural_network import MLPClassifier
from scipy.optimize import curve_fit
import datetime
from brokenaxes import brokenaxes
nan=float('nan')
def warn(*args, **kwargs):
    pass
plt.close('all')



Let's define some functions that will be used later

In [3]:
# Make pretty plots
def adjust_spines(ax, spines):
    for loc, spine in ax.spines.items():
        if loc in spines: 
            if loc=='left':
                spine.set_position(('outward', 10))  # outward by 10 points
            if loc=='bottom':
                spine.set_position(('outward', 0))  # outward by 10 points
         #   spine.set_smart_bounds(True)
        else:
            spine.set_color('none')  # don't draw spine
    # turn off ticks where there is no spine
    if 'left' in spines:
        ax.yaxis.set_ticks_position('left')
    else:
        # no yaxis ticks
        ax.yaxis.set_ticks([])
    if 'bottom' in spines:
        ax.xaxis.set_ticks_position('bottom')
    else:
        # no xaxis ticks
        ax.xaxis.set_ticks([])

# Sigmoid curve
def log_curve(x,a,c):
    num=1+np.exp(-a*(x+c))
    return 1/num

In [ ]:
# Parameters to fit the psychometric curve
maxfev=100000
p00_psy=(0.2,0)
p01_psy=(0.2,-3)
p02_psy=(0.2,3)
method='lm'

group_coh=np.array([-7 ,-6 ,-5 ,-4 ,-3 ,-2 ,-1 ,0  ,1  ,2  ,3  ,4  ,5  ,6  ,7])

xx_coh=np.array([-51.2,-25.6,-12.8,-6.4,-3.2,-1.6,0,1.6,3.2,6.4,12.8,25.6,51.2])
xx_plot=np.array(['-51.2','-25.6','-12.8','-6.4','-3.2','-1.6','0','1.6','3.2','6.4','12.8','25.6','51.2'])
ind_uu=np.array([0,1,2,3,5,6,7,8,9,11,12,13,14]) # missing 4 and 10

n_gen=1000
xx_fit=np.linspace(xx_coh[0],xx_coh[-1],n_gen)

psycho=nan*np.zeros((len(files),len(xx_coh),3))
fit_psycho=nan*np.zeros((len(files),n_gen,3))

data=scipy.io.loadmat('../data_demo/N20170204b.FIRA.dots_rt_rew_imbalance.PAG.mat',struct_as_record=False,simplify_cells=True)


In [ ]:
beha=miscellaneous.behavior(data,group_coh)
change_ctx=beha['change_ctx']
ind_ch=np.where(change_ctx!=0)[0]
index_nonan=beha['index_nonan']
reward=beha['reward']
coh_signed=beha['coherence_signed']
coh_set_signed=np.unique(coh_signed)

coh_set_signed=coh_set_signed[1:-1]
context=beha['context']
stimulus=beha['stimulus']
choice=beha['choice']

# Psychometric curve
for i in range(len(coh_set_signed)):
    ind_coh=np.where(coh_signed==coh_set_signed[i])[0]
    ind_ct0=np.where(context[ind_coh]==0)[0]
    ind_ct1=np.where(context[ind_coh]==1)[0]
    psycho[kk,i,0]=np.mean(choice[ind_coh])
    psycho[kk,i,1]=np.mean(choice[ind_coh][ind_ct0])
    psycho[kk,i,2]=np.mean(choice[ind_coh][ind_ct1])

# Fit Psychometric
popt0,pcov0=curve_fit(log_curve,100*coh_set_signed,psycho[kk,:,0],maxfev=maxfev,p0=p00_psy,method=method)
fit_psycho[kk,:,0]=log_curve(xx_fit,popt0[0],popt0[1])
params_psy[kk,:,0]=popt0
#params_psy_all[kk,:,0]=popt0
popt1,pcov1=curve_fit(log_curve,100*coh_set_signed,psycho[kk,:,1],maxfev=maxfev,p0=p01_psy,method=method)
fit_psycho[kk,:,1]=log_curve(xx_fit,popt1[0],popt1[1])
params_psy[kk,:,1]=popt1
popt2,pcov2=curve_fit(log_curve,100*coh_set_signed,psycho[kk,:,2],maxfev=maxfev,p0=p02_psy,method=method)
fit_psycho[kk,:,2]=log_curve(xx_fit,popt2[0],popt2[1])
params_psy[kk,:,2]=popt2
